```sh
ncu --section=SpeedOfLight -o$model sh -c \
    "echo | trt/exec -s'\"input_tensor:0\":1x3x224x224' $model.engine"
ncu --csv -i$model.ncu-rep >$model.csv
```

In [19]:
import matplotlib.pyplot as plt
import numpy as np

import csv
import locale

locale.setlocale(locale.LC_ALL, '')

for model in ['resnet', 'bert']:
    durations = []
    throughputs = []
    with open(model + '.csv', newline='') as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row['Section Name'] != 'GPU Speed Of Light Throughput': continue
            k = row['Metric Name']
            v = locale.atof(row['Metric Value'])
            if k == 'Duration':
                durations.append(v)
            if not k.endswith(' Throughput'): continue
            i = int(row['ID'])
            if len(throughputs) <= i:
                throughputs.append([])
                assert len(throughputs) > i
            throughputs[i].append(v)
    x = np.cumsum(np.array(durations))
    labels = [
        'memory',
        'DRAM',
        'L1/TEX cache',
        'L2 cache',
        'compute (SM)'
    ]
    plt.plot(x, np.array(throughputs), label=labels)
    plt.title(model)
    plt.legend()
    plt.savefig(model + '.png')
    plt.clf()

<Figure size 640x480 with 0 Axes>